In [ ]:
import tensorflow as tf
import numpy as np
from copy import deepcopy
from tensorflow.examples.tutorials.mnist import input_data
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from IPython import display

In [ ]:
learning_rate = 0.001
training_iters = 200000
batch_size = 100
disp_freq = 100

n_input = 784 # MNIST data input (img shape: 28*28)
n_classes = 10 # MNIST total classes (0-9 digits)
dropout = 0.75

x = tf.placeholder(tf.float32, [None, n_input])
y_ = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [ ]:

def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)




# Create model
def conv_net(x, weights, biases):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

class Model:


    def __init__(self, x, y_):

#         self.x = x # input placeholder
    
#         weights = {
#     # 5x5 conv, 1 input, 32 outputs
#         'wc1': tf.Variable(tf.truncated_normal([5, 5, 1, 32],stddev=0.001)),
#         # 5x5 conv, 32 inputs, 64 outputs
#         'wc2': tf.Variable(tf.truncated_normal([5, 5, 32, 64],stddev=0.001)),
#         # fully connected, 7*7*64 inputs, 1024 outputs
#         'wd1': tf.Variable(tf.truncated_normal([7*7*64, 1024],stddev=0.001)),
#         # 1024 inputs, 10 outputs (class prediction)
#         'out': tf.Variable(tf.truncated_normal([1024, n_classes],stddev=0.001))
#         }

#         biases = {
#         'bc1': tf.Variable(tf.truncated_normal([32],stddev=0.001)),
#         'bc2': tf.Variable(tf.truncated_normal([64],stddev=0.001)),
#         'bd1': tf.Variable(tf.truncated_normal([1024],stddev=0.001)),
#         'out': tf.Variable(tf.truncated_normal([n_classes],stddev=0.001))
#         }
       
        

#         fc1 = tf.nn.dropout(fc1, dropout)
#         self.y = conv_net(x, weights, biases)

#         self.var_list=[weights['wc1'],weights['wc2'],weights['wd1'],weights['out'],
#                        biases['bc1'],biases['bc2'],biases['bd1'],biases['out']]


#         
        
#         
#         self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.y,labels=y_))
#         self.ewc_loss = self.cross_entropy
#         self.train_step = tf.train.AdamOptimizer(0.001).minimize(self.ewc_loss)
        
#         correct_prediction = tf.equal(tf.argmax(self.y,1), tf.argmax(y_,1))
#         self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))


        in_dim = int(x.get_shape()[1]) # 784 for MNIST
        out_dim = int(y_.get_shape()[1]) # 10 for MNIST

        self.x = x # input placeholder

        # simple 2-layer network
        W1 = weight_variable([in_dim,50])
        b1 = bias_variable([50])

        W2 = weight_variable([50,out_dim])
        b2 = bias_variable([out_dim])

        h1 = tf.nn.relu(tf.matmul(x,W1) + b1) # hidden layer
        self.y = tf.matmul(h1,W2) + b2 # output layer

        self.var_list = [W1, b1, W2, b2]
        
       
        # vanilla single-task loss
        self.cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=self.y))
        if not hasattr(self, "ewc_loss"):
            print "origin cross entropy"
            self.ewc_loss = self.cross_entropy
        self.train_step = tf.train.GradientDescentOptimizer(0.01).minimize(self.ewc_loss)
        # performance metrics
        correct_prediction = tf.equal(tf.argmax(self.y,1), tf.argmax(y_,1))
        self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        
    def compute_fisher(self, imgset, sess, num_samples=200, plot_diffs=False, disp_freq=10):
        # computer Fisher information for each parameter

        # initialize Fisher information for most recent task
        self.F_accum = []
        for v in range(len(self.var_list)):
            self.F_accum.append(np.zeros(self.var_list[v].get_shape().as_list()))

        # sampling a random class from softmax
        probs = tf.nn.softmax(self.y)
        class_ind = tf.to_int32(tf.multinomial(tf.log(probs), 1)[0][0])

        if(plot_diffs):
            # track differences in mean Fisher info
            F_prev = deepcopy(self.F_accum)
            mean_diffs = np.zeros(0)

        for i in range(num_samples):
            # select random input image
            im_ind = np.random.randint(imgset.shape[0])
            # compute first-order derivatives
            ders = sess.run(tf.gradients(tf.log(probs[0,class_ind]), self.var_list), feed_dict={self.x: imgset[im_ind:im_ind+1]})
            # square the derivatives and add to total
            for v in range(len(self.F_accum)):
                self.F_accum[v] += np.square(ders[v])
            if(plot_diffs):
                if i % disp_freq == 0 and i > 0:
                    # recording mean diffs of F
                    F_diff = 0
                    for v in range(len(self.F_accum)):
                        F_diff += np.sum(np.absolute(self.F_accum[v]/(i+1) - F_prev[v]))
                    mean_diff = np.mean(F_diff)
                    mean_diffs = np.append(mean_diffs, mean_diff)
                    for v in range(len(self.F_accum)):
                        F_prev[v] = self.F_accum[v]/(i+1)
                    plt.plot(range(disp_freq+1, i+2, disp_freq), mean_diffs)
                    plt.xlabel("Number of samples")
                    plt.ylabel("Mean absolute Fisher difference")
                    display.display(plt.gcf())
                    display.clear_output(wait=True)

        # divide totals by number of samples
        for v in range(len(self.F_accum)):
            self.F_accum[v] /= num_samples
    
    def set_vanilla_loss(self):
        self.train_step = tf.train.GradientDescentOptimizer(0.01).minimize(self.cross_entropy)

    def star(self):
    # used for saving optimal weights after most recent task training
        self.star_vars = []
        for v in range(len(self.var_list)):
            self.star_vars.append(self.var_list[v].eval())
    def update_ewc_loss(self, lam):
        # elastic weight consolidation
        # lam is weighting for previous task(s) constraints

        if not hasattr(self, "ewc_loss"):
            self.ewc_loss = self.cross_entropy

        for v in range(len(self.var_list)):
            self.ewc_loss += (lam/2) * tf.reduce_sum(tf.multiply(self.F_accum[v].astype(np.float32),tf.square(self.var_list[v] - self.star_vars[v])))
       
    def restore(self, sess):
    # reassign optimal weights for latest task
        if hasattr(self, "star_vars"):
            for v in range(len(self.var_list)):
                sess.run(self.var_list[v].assign(self.star_vars[v]))

In [ ]:
model = Model(x, y_)
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

In [ ]:
def mnist_imshow(img):
    plt.imshow(img.reshape([28,28]), cmap="gray")
    plt.axis('off')

def permute_mnist(mnist):
    perm_inds = range(mnist.train.images.shape[1])
    np.random.shuffle(perm_inds)
    mnist2 = deepcopy(mnist)
    sets = ["train", "validation", "test"]
    for set_name in sets:
        this_set = getattr(mnist2, set_name) # shallow copy
        this_set._images = np.transpose(np.array([this_set.images[:,c] for c in perm_inds]))
    return mnist2

def plot_test_acc(plot_handles):
    plt.legend(handles=plot_handles, loc="center right")
    plt.xlabel("Iterations")
    plt.ylabel("Test Accuracy")
    plt.ylim(0,1)
    display.display(plt.gcf())
    display.clear_output(wait=True)



In [ ]:
def train_task(model, num_iter, disp_freq, trainset, testsets, x, y_, lams=[0]):
    for l in range(len(lams)):
        # lams[l] sets weight on old task(s)
        model.restore(sess) # reassign optimal weights from previous training session
        if(lams[l] != 0):
            #model.set_vanilla_loss()
        #else:
            model.update_ewc_loss(lams[l])
        # initialize test accuracy array for each task 
        test_accs = []
        for task in range(len(testsets)):
            test_accs.append(np.zeros(num_iter/disp_freq))
        # train on current task
        step = 0
        for iter in range(num_iter):
            batch = trainset.train.next_batch(batch_size)
            model.train_step.run(feed_dict={x: batch[0], y_: batch[1]})
            #print model.prediction.eval(feed_dict={x: mnist.test.images})
            if (iter % disp_freq == 0):
                plt.subplot(1, len(lams), l+1)
                plots = []
                colors = ['r', 'b', 'g']
                for task in range(len(testsets)):
                    feed_dict={x: testsets[task].test.images, y_: testsets[task].test.labels}
                    test_accs[task][iter/disp_freq] = model.accuracy.eval(feed_dict=feed_dict)
                    c = chr(ord('A') + task)
                    plot_h, = plt.plot(range(1,iter+2,disp_freq), test_accs[task][:iter/disp_freq+1], colors[task], label="task " + c)
                    plots.append(plot_h)
                plot_test_acc(plots)
                if l == 0: 
                    plt.title("sgd")
                else:
                    plt.title("ewc")
                plt.gcf().set_size_inches(len(lams)*5, 3.5)

In [ ]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())
train_task(model, 2000, 100, mnist, [mnist], x, y_, lams=[0])

In [ ]:
model.compute_fisher(mnist.validation.images, sess, num_samples=200, plot_diffs=True)

In [ ]:
mnist2 = permute_mnist(mnist)

plt.subplot(1,2,1)
mnist_imshow(mnist.train.images[5])
plt.title("original task image")
plt.subplot(1,2,2)
mnist_imshow(mnist2.train.images[5])
plt.title("new task image");

In [ ]:
model.star()
train_task(model, 1200, 20, mnist2, [mnist, mnist2], x, y_, lams=[0, 15])

In [ ]:
model.compute_fisher(mnist2.validation.images, sess, num_samples=200, plot_diffs=True)

In [ ]:
mnist3 = permute_mnist(mnist)

In [ ]:
model.star()
train_task(model, 1200, 20, mnist3, [mnist, mnist2, mnist3], x, y_, lams=[0, 15])